In [1]:
import cartopy.crs as ccrs
import cartopy.feature as cf
import matplotlib.pyplot as plt
import matplotlib
from matplotlib import cm
from matplotlib import colors
import xarray as xr
import numpy as np
from matplotlib.ticker import FuncFormatter
from matplotlib.gridspec import GridSpec
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

In [ ]:
basedir = '/input/directory/'
#
IMERGE = basedir + 'Obs_GPM_IMERG_pr_20160809-20160909_Asia_timeavg.nc'
IMERGE = xr.open_dataset( IMERGE,decode_times=False)
IMERGE_vals_pr = IMERGE['precipitationCal']
lons_IMERGE = IMERGE.lon
lats_IMERGE = IMERGE.lat
#
ERA5_cwp = basedir + 'Obs_ERA5_cwp_20160809-20160909_Asia_timeavg.nc'
ERA5_cwp = xr.open_dataset( ERA5_cwp)
ERA5_vals_cwp = ERA5_cwp['tciw']
lons_ERA5_cwp = ERA5_cwp.longitude
lats_ERA5_cwp = ERA5_cwp.latitude

def resize_colorbar(event):
    plt.draw()

    posn = ax.get_position()
    cbar_ax.set_position([posn.x0 + posn.width + 0.01, posn.y0,
                          0.04, posn.height])

levs_pr = np.arange(0,2,0.05)
levs_cwp = np.arange(0,0.5,0.05)


class MidpointNormalize(colors.Normalize):
        """
        Normalise the colorbar so that diverging bars work there way either side from a prescribed midpoint value)

        e.g. im=ax1.imshow(array, norm=MidpointNormalize(midpoint=0.,vmin=-100, vmax=100))
        """
        def __init__(self, vmin=None, vmax=None, midpoint=None, clip=False):
                self.midpoint = midpoint
                colors.Normalize.__init__(self, vmin, vmax, clip)

        def __call__(self, value, clip=None):
                x, y = [self.vmin, self.midpoint, self.vmax], [0, 0.5, 1]
                return np.ma.masked_array(np.interp(value, x, y), np.isnan(value))

def fmt( x, pos ):
    a, b = '{:.1e}'.format(x).split('e')
    b = int(b)
    return r'${} \times 10^{{{}}}$'.format(a, b)

# Colorbar with NSW Precip colors
nws_precip_colors = [
    "#fdfdfd",  # 0.01 - 0.10 inches
    "#E6E6FA",
    "#DDA0DD",  # 0.10 - 0.25 inches
    "#C79FEF",  # 0.25 - 0.50 inches
    "#7BC8F6",  # 0.50 - 0.75 inches
    "#0343DF",  # 0.75 - 1.00 inches
    "#7FFFD4",  # 1.00 - 1.50 inches
    "#AAFF32",  # 1.50 - 2.00 inches
    "#C1F80A",  # 2.00 - 2.50 inches
    "#F97306",  # 2.50 - 3.00 inches
    "#FBDD7E",  # 3.00 - 4.00 inches
    "#FFA500",  # 4.00 - 5.00 inches
]
precip_colormap = matplotlib.colors.ListedColormap(nws_precip_colors)
#
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(20, 7.5), subplot_kw={'projection': ccrs.PlateCarree()})
plt.subplots_adjust(left=0.01, right=0.95, bottom=0.05, top=0.8, wspace=0, hspace=0.1)
gs = GridSpec(1, 2, figure=fig)
im = axes[0].contourf(lons_IMERGE, lats_IMERGE, IMERGE_vals_pr[0], levels=levs_pr, cmap=plt.cm.rainbow, extend='both')
axes[0].set_title(r'$\bf{(a)}$ GPM IMERG', size=30)
axes[0].coastlines()
axes[0].add_feature( cf.BORDERS )
gl = axes[0].gridlines( crs=ccrs.PlateCarree(), draw_labels=True, linewidth=1, color='gray', alpha=0.5, linestyle='--' )
gl.top_labels = False
gl.right_labels = False
gl.bottom_labels = True
gl.left_labels = True
gl.xlabel_style = { 'size':30, 'rotation':35  }
gl.ylabel_style = { 'size':30, 'rotation':35 }
#
cbar_ax = fig.add_axes([0.44, 0.08, 0.012, 0.70]) # [left, bottom, width, height]
im = axes[0].contourf(lons_IMERGE, lats_IMERGE, IMERGE_vals_pr[0],levels=levs_pr, transform=ccrs.PlateCarree(), cmap=precip_colormap2)
cbar = plt.colorbar(im, cax=cbar_ax, shrink=0.8, aspect=20)
cbar.set_label("mm.hr$^{-1}$", size=25)
cbar.ax.tick_params(labelsize=25)
#
im = axes[1].contourf(lons_IMERGE_cwp, lats_IMERGE_cwp, IMERGE_vals_cwp[0], levels=levs_cwp, cmap=precip_colormap, extend='both')
axes[1].set_title(r'$\bf{(b)}$ ERA5 CWP', size=30)
axes[1].coastlines()
axes[1].add_feature( cf.BORDERS )
#
#
gl = axes[1].gridlines( crs=ccrs.PlateCarree(), draw_labels=True, linewidth=1, color='gray', alpha=0.5, linestyle='--' )
gl.top_labels = False
gl.right_labels = False
gl.bottom_labels = True
gl.left_labels = False
gl.xlabel_style = { 'size':30, 'rotation':35  }
gl.ylabel_style = { 'size':30, 'rotation':35 }
#
cbar_ax = fig.add_axes([0.91, 0.09, 0.012, 0.70])
im = axes[1].contourf(lons_IMERGE_cwp, lats_IMERGE_cwp, IMERGE_vals_cwp[0], levels=levs_cwp, transform=ccrs.PlateCarree(), cmap=precip_colormap)
cbar = plt.colorbar(im, cax=cbar_ax, shrink=0.8, aspect=20)
cbar.set_label(r'[kgm$^{-2}$]', size=25)
cbar.ax.tick_params(labelsize=25)
#
plt.savefig('/output/directory/IMERGE_CWP.png', dpi=200, bbox_inches='tight')
plt.savefig("/output/directory/IMERGE_CWP.pdf", format="pdf",dpi=200, bbox_inches="tight")